In [77]:
import pandas as pd
import numpy as np
import os
import json
# import gui
from collections import Counter
import re
import math as m

pd.options.display.max_columns = 20000

# Create a pandas dataframe of all cards
![Alt Text](https://media.giphy.com/media/vFKqnCdLPNOKc/giphy.gif)

### Create initial dataframe and perform some preprocessing tasks

In [78]:
lexicon = pd.read_feather('mtg_card_database')
colors = ["W", "U", "B", "R", "G", "C"]

In [79]:
# Only keep the most recent arena set, dropping all reprints
lexicon = lexicon.sort_values(by='arena_id', ascending=False).copy()
lexicon = lexicon.drop_duplicates(subset=["name"])

# Set the index for card names to easily search
def set_index(lexicon):
    lexicon = lexicon.set_index("name", drop=False)
    lexicon.index.name = "card_name"
    return lexicon
lexicon = set_index(lexicon)

# Drop unused columns
lexicon = lexicon.drop(columns=['id', 'power', 'toughness', 'color_identity', 'set', 'keywords', 'flavor_text', 'edhrec_rank', 'arena_id', 'loyalty'], inplace=False)

In [80]:
# Clean mana_cost
def clean_mana_cost(s):
    """Cleans the mana_cost column by removing curly braces {} and by converting any numbers in mana cost to 1s representation

    Args:
        s (string): the mana_cost string 

    Returns:
        string: cleaned mana cost
    """
    # If s is not nonetype
    try: 
        s = s.replace("{", "").replace("}", "")

        # If the first character of s is a number
        try:
            num = int(s[0])
            s = s.replace(s[0], '1' * num)
        except:
            pass
    except:
        pass

    return s

lexicon['mana_cost'] = lexicon['mana_cost'].apply(lambda s: clean_mana_cost(s))

## Fix Dual-Faced Cards

### Extract card data for both sides

In [81]:
# Let's first create a dataframe that just has the card name and the column 'card_faces'
double_cards_df = lexicon[['name','card_faces']].dropna()

# DFC are a list of two dictionaries [ {k:v, k2:v2}, {k3:v3} ]
def split_card_faces(row, face_number):
    face = row[face_number]
    return face

face_one = 0
face_two = 1
double_cards_df['face1'] = double_cards_df['card_faces'].apply(lambda row: split_card_faces(row, face_one))
double_cards_df['face2'] = double_cards_df['card_faces'].apply(lambda row: split_card_faces(row, face_two))

# # Now let's drop the column 'card_faces'
# double_cards_df = double_cards_df.copy()
# double_cards_df.drop("card_faces",axis=1)

# We now go into each key within the dictionary of face1 and face2 and separate them into columns
try:
    double_cards_df[double_cards_df['face1'].apply(pd.Series).columns + "_1"] = double_cards_df['face1'].apply(pd.Series)
    double_cards_df[double_cards_df['face2'].apply(pd.Series).columns + "_2"] = double_cards_df['face2'].apply(pd.Series)
except:
    pass

# Define a list of columns we want to keep from the DFCs
cols_to_keep = ['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
                'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']

# For each column in the dataframe, if it's not a selected column, we drop it
for col in double_cards_df.columns:
    if col not in cols_to_keep:
        double_cards_df.drop(col, axis=1, inplace=True)

# We now need to consolidate the 2 oracle texts into 1, we join them together
double_cards_df['oracle_text_dobles'] = double_cards_df['oracle_text_1'] + "\n" + double_cards_df['oracle_text_2']

# Clean the mana costs
double_cards_df['mana_cost_1'] = double_cards_df['mana_cost_1'].apply(lambda s: clean_mana_cost(s))
double_cards_df['mana_cost_2'] = double_cards_df['mana_cost_2'].apply(lambda s: clean_mana_cost(s))

In [82]:
# We now merge them by card name
lexicon = lexicon.merge(double_cards_df, on=["name"], how="left")

In [83]:
# If oracle_text is empty (meaning it's a double faced card), we replace it with our 'oracle_text_dobles' column
lexicon['oracle_text'] = np.where(lexicon['oracle_text'].isna(),lexicon['oracle_text_dobles'],lexicon['oracle_text'])


In [84]:
# Drop secret lair dual faced cards which should be single faced cards
tmpdf = lexicon[lexicon['name'].str.contains("//")]
mask = tmpdf[tmpdf['type_line'].isna()]
lexicon = lexicon[~lexicon['name'].isin(mask['name'])]

In [85]:
# We use this script to replace Nulls with "None"
# lexicon[['oracle_text_1','oracle_text_2']] = lexicon[['oracle_text_1','oracle_text_2']]

# try:
#     lexicon[['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
#                 'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']] = lexicon[['name', 'name_1', 'name_2', 'oracle_text_1','oracle_text_2',
#                                                                                         'mana_cost_1', 'mana_cost_2', 'type_line_1', 'type_line_2']]
# except:
#     pass

# Now that we have our oracle text from the 2 card sides joined together, we want to use it to replace
# the actual "oracle_text" from the original dataframe, which is actually empty

# lexicon = set_index(lexicon)

# # And now that column is useless so we drop it
# lexicon = lexicon.drop("oracle_text_dobles",axis=1)

### Average the mana cost of both sides as well as the cmc

In [86]:
def dfc_avg_mana_cost_and_cmc(row):
    """Averages the mana cost for dual-faced cards and the cmc

    Args:
        row (pd.Series): one row of the lexicon df

    Returns:
        tuple of strings: averaged mana cost and average cmc
    """
    # Only for dual faced cards
    if isinstance(row.card_faces, type(np.ndarray(0))):
        # Initialize return value
        mana_cost = ''
        # Only for DFC with mana costs
        try:
            # Count the values of each mana symbol and add them together into a dictionary
            s1 = Counter(row.mana_cost_1)
            s2 = Counter(row.mana_cost_2)
            s3 = s1+s2
            if s3 == s1:
                row.mana_cost = row.mana_cost_1
                return row
            elif s3 == s2:
                row.mana_cost = row.mana_cost_2
                return row
            s4 = dict(s3)
            # Divide all the values by 2 rounded up for Mana symbols and down for generic costs
            for k in s4.keys():
                # If there is a generic casting cost
                try:
                    int(k)
                    s4[k] = max(m.floor(s4[k] / 2), 1)
                # If there is a colored mana symbol
                except:
                    s4[k] = m.ceil(s4[k] / 2)
                # concatenate the dict back into a string
                mana_cost += (k * s4[k])
            # Average cmc which is a return value
            avg_cmc = sum(s3.values()) / 2
            row.cmc = avg_cmc
            row.mana_cost = mana_cost
            # Return the averaged mana cost as a string
            return row
        except:
            pass
    # If its not a DFC then just return the mana_cost already present
    return row

lexicon = lexicon.apply(dfc_avg_mana_cost_and_cmc, axis=1)

### Create columns for casting cost pips

In [87]:
def count_pips(mana_cost):
    """Count the number of pips in each color for a given mana cost

    Args:
        mana_cost (str): string representation of the mana cost

    Returns:
        dict: number of pips for each color
    """
    pips = Counter(mana_cost)
    result = {f'cast_cost_{color}': pips[f'{color}'] for color in colors}
    return result

pips = lexicon['mana_cost'].apply(count_pips)
pips_df = pd.DataFrame(list(pips))
lexicon = pd.concat([lexicon, pips_df], axis=1)


In [71]:
lexicon[lexicon['name'] == "Kozilek, Butcher of Truth"]

,name,mana_cost,cmc,type_line,oracle_text,colors,produced_mana,rarity,card_faces,mana_cost_1,name_1,oracle_text_1,type_line_1,mana_cost_2,name_2,oracle_text_2,type_line_2,oracle_text_dobles,cast_cost_W,cast_cost_U,cast_cost_B,cast_cost_R,cast_cost_G,cast_cost_C
9052,"Kozilek, Butcher of Truth",10,10.0,Legendary Creature — Eldrazi,"When you cast this spell, draw four cards.\nAn...",[],None,mythic,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0


In [76]:
def count_pips(row):
    """Applied to the mana_cost column in lexicon to create columns for the number of pips in that color for each color

    Args:
        row (pd.Series): row in lexicon

    Returns:
        pd.Series: new row with created casting cost pip columns
    """
    pips = Counter(row.mana_cost)

    for color in colors:
        row_name = 'cast_cost_{}'.format(color)
        row[f'{row_name}'] = pips[f'{color}']

    return row

lexicon = lexicon.apply(count_pips, axis=1)

# print(count_pips("WUBRG")["R"])
lexicon

TypeError: 'float' object is not iterable

In [277]:
# A copy of the lexicon df is created so that we can avoid making changes to a copy of a sliced df
lexicon = lexicon.copy()
lexicon.loc[:, 'cast_cost_W'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['W'], na_action='ignore')
lexicon.loc[:, 'cast_cost_U'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['U'], na_action='ignore')
lexicon.loc[:, 'cast_cost_B'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['B'], na_action='ignore')
lexicon.loc[:, 'cast_cost_R'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['R'], na_action='ignore')
lexicon.loc[:, 'cast_cost_G'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['G'], na_action='ignore')
lexicon.loc[:, 'cast_cost_C'] = lexicon['mana_cost'].map(lambda s: count_pips(s)['C'], na_action='ignore')


### Create columns for produced mana colors of non-land cards

In [279]:
def count_produced_mana(s):
    """Applied to the produced_mana column, returns the colors produced by a card

    Args:
        s (List[str]): 

    Returns:
        prod_mana: _description_
    """    
    return Counter(s)

print(count_produced_mana(['B']))

Counter({'B': 1})


In [280]:
lexicon.loc[:, 'produces_W'] = lexicon['produced_mana'].map(lambda s: count_produced_mana(s)['W'], na_action='ignore')
lexicon.loc[:, 'produces_U'] = lexicon['produced_mana'].map(lambda s: count_produced_mana(s)['U'], na_action='ignore')
lexicon.loc[:, 'produces_B'] = lexicon['produced_mana'].map(lambda s: count_produced_mana(s)['B'], na_action='ignore')
lexicon.loc[:, 'produces_R'] = lexicon['produced_mana'].map(lambda s: count_produced_mana(s)['R'], na_action='ignore')
lexicon.loc[:, 'produces_G'] = lexicon['produced_mana'].map(lambda s: count_produced_mana(s)['G'], na_action='ignore')
lexicon.loc[:, 'produces_C'] = lexicon['produced_mana'].map(lambda s: count_produced_mana(s)['C'], na_action='ignore')


- Mana_Costs
    - Phyrexian Mana
    - Dual Color (either red or white mana can be used to pay for RW)
    - X cost (average value for X is math.floor(4 / num_x))
- Mana Cost Pips Optimizing
- Card Selection
    - define RegEx pattern
    - Create method
- Cards which make treasure
- Beautiful Soup
    - How to use it
    - How big do we want our dataset
    - How do we store the dataframes???